- Time to concatenate 24 hours of LFP (12x 2hr files) = 30 minutes. 
- Size of 24 hours of LFP = ~160 GB

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import subprocess
from pathlib import Path
from datetime import datetime
from joblib import Parallel, delayed
from ecephys.sglx_utils.cat_gt import get_catGT_command

In [ ]:
def get_run_specs(subject, condition_group):
    from ecephys_analyses.data import paths
    from ecephys_analyses.data.channel_groups import full_names
    from ecephys.data.paths import parse_sglx_stem
    conditions = paths.get_conditions(subject, condition_group)
    datapath_dict = paths.load_datapath_yaml()
    all_run_specs = []
    for cond in conditions:
        cond_spec = datapath_dict[subject][cond]
        # Name
        full_name = full_names[subject]
        # exp
        assert len(cond_spec.keys()) == 1
        exp = list(cond_spec.keys())[0]
        # run
        stems = cond_spec[exp]
        assert len(stems) == 1
        run, gate, trig, probe = parse_sglx_stem(stems[0])
        all_run_specs.append(
            tuple([full_name, exp, run, {
                'g': gate.split('g')[1],
                'prb': probe.split('imec')[1],
                't': trig.split('t')[1],
            }])
        )
    return all_run_specs

In [ ]:
# ----------
# USER

analysis_id = 'tom/catgt'

catgt_base_cfg = {
    'aphipass': 300,
    'aplopass': 9000,
    'gbldmx': True,
    'gfix': '0.40,0.10,0.02',
}  # Applied to all

sync_channel = 384  # Applied to all

###
####
######## Manual
# Subject, experiment, run, catgt_cfg
# catGT_cfg must contains the following keys: run, t, g, prb, 
# all_run_specs = [
#     ('CNPIX8-Allan', '3-5-2021', '3-5-2021',    {'g': '2', 'prb': '0', 't': '0,0',}), 
#     ('CNPIX5-Alessandro', '8-31-2020', '8-31-2020_REMStim',    {'g': '0', 'prb': '0', 't': '0,0',}), 
# ]
########## Auto
subject, condition_group = 'Eugene', 'eStim'
all_run_specs = get_run_specs(subject, condition_group)
print(f"N runs = {len(all_run_specs)}")
print(f"run specs (first 5). ```subject, experiment, run, catgt_cfg``` Looks good?: {all_run_specs[0:5]}")
####
####
####


ap = True
lf = False

n_jobs = 5

dry_run=False  # Creates output dirs, doesn't run catGT

# END USER
# ----------

In [ ]:
ANALYSIS_ROOT = '/Volumes/neuropixel/Data/'
RAW_DATA_ROOT = '/Volumes/neuropixel_archive/Data/chronic'

In [ ]:
CATGT_PATH = "/Volumes/scratch/neuropixels/bin/CatGT-linux/runit.sh"
WINE_PATH = None

In [ ]:
def run_catgt(run_specs, dry_run=dry_run):
    subj_id, exp_id, run_id, run_catgt_cfg = run_specs
    
    catgt_cfg = {
        **run_catgt_cfg,
        **catgt_base_cfg,
    }
    
    for key in ['g', 't', 'prb']:
        assert key in catgt_cfg
    
    src_dir = Path(RAW_DATA_ROOT)/subj_id/exp_id
    dest_dir = Path(ANALYSIS_ROOT)/analysis_id/subj_id/exp_id
    dest_dir.mkdir(parents=True, exist_ok=True)
    
    cmd = get_catGT_command(
        catGT_path=CATGT_PATH,
        wine_path=WINE_PATH,
        dir=str(src_dir),
        dest=str(dest_dir),
        run=run_id,
        ap=ap,
        lf=lf,
        SY=f"{catgt_cfg['prb']},{sync_channel},6,500",
        prb_fld=True,
        out_prb_fld=True,
        **catgt_cfg,
    )
    

    start = datetime.now()
    print(f"Running {cmd}")
    if not dry_run:
        process = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    end = datetime.now()

    print(f"{end.strftime('%H:%M:%S')}: Finished {subj_id}, {run_id}. Run time = {str(end - start)}")

In [ ]:
if n_jobs == 1:
    for run_specs in all_run_specs:
        print(run_specs)
        run_catgt(run_specs)
else:
    parallel = Parallel(
        n_jobs=n_jobs,
        backend='multiprocessing',
    )(delayed(run_catgt)(run_specs) for run_specs in all_run_specs)